In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\ravis\OneDrive\Desktop\LogClassification\training\dataset\synthetic_logs.csv")


In [2]:
df.columns # type: ignore

Index(['timestamp', 'source', 'log_message', 'target_label', 'complexity'], dtype='object')

In [3]:
df.source.unique() # type: ignore

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN


model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['log_message'].tolist(), show_progress_bar=True)

print(embeddings.shape)



c:\Users\ravis\OneDrive\Desktop\LogClassification\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 76/76 [00:30<00:00,  2.49it/s]

(2410, 384)


In [8]:
dbscan = DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

df['cluster'] = clusters
df['cluster'] = df['cluster'].astype(str)


In [10]:
df.head() # type: ignore

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0


In [ ]:
df[df['cluster'] == '1']

In [16]:
cluster_counts = df['cluster'].value_counts()
large_clusters = cluster_counts[cluster_counts > 10].index.tolist()

for cluster in large_clusters:
    print(f"Cluster {cluster} has {cluster_counts[cluster]} logs")
    print(df[df['cluster'] == cluster]['log_message'].head(5).to_string(index=False))
    print("\n---\n")

Cluster 0 has 1017 logs
nova.osapi_compute.wsgi.server [req-b9718cd8-f6...
nova.osapi_compute.wsgi.server [req-4895c258-b2...
nova.osapi_compute.wsgi.server [req-ee8bc8ba-92...
nova.osapi_compute.wsgi.server [req-f0bffbc3-5a...
nova.osapi_compute.wsgi.server [req-2bf7cfee-a2...

---

Cluster 5 has 147 logs
nova.compute.claims [req-a07ac654-8e81-416d-bfb...
nova.compute.claims [req-d6986b54-3735-4a42-907...
nova.compute.claims [req-72b4858f-049e-49e1-b31...
nova.compute.claims [req-5c8f52bd-8e3c-41f0-95a...
nova.compute.claims [req-d38f479d-9bb9-4276-968...

---

Cluster 11 has 100 logs
User User685 logged out.
 User User395 logged in.
 User User225 logged in.
User User494 logged out.
 User User900 logged in.

---

Cluster 13 has 86 logs
Backup started at 2025-05-14 07:06:55.
Backup started at 2025-02-15 20:00:19.
  Backup ended at 2025-08-08 13:06:23.
Backup started at 2025-11-14 08:27:43.
Backup started at 2025-12-09 10:19:11.

---

Cluster 7 has 60 logs
Multiple bad login attempts de

In [27]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
    return None


In [28]:
classify_with_regex("Disk cleanup completed successfully.")

'System Notification'

In [29]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)

In [33]:
df[df.regex_label.notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,4,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,8,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,9,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,21,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,13,System Notification


In [34]:
df_non_regex = df[df.regex_label.isnull()].copy(deep=True)

df_non_regex.shape

(1910, 7)

In [36]:
type(df_non_regex['target_label'].value_counts())

pandas.core.series.Series

In [42]:
target_label_counts = df_non_regex['target_label'].value_counts()
rare_labels = target_label_counts[target_label_counts <= 5].index.tolist()
print(rare_labels)

['Workflow Error', 'Deprecation Warning']


In [43]:
df_non_legacy = df_non_regex[~df_non_regex['target_label'].isin(rare_labels)].copy(deep=True)

In [44]:
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [45]:
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 60/60 [00:28<00:00,  2.09it/s]


In [50]:
X = filtered_embeddings
y = df_non_legacy['target_label'].to_numpy(dtype=str)

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [58]:
import os
import joblib

os.makedirs('training/models', exist_ok=True)
joblib.dump(clf, 'training/models/log_classifier.joblib')

['training/models/log_classifier.joblib']